# Installation

In [ ]:
!pip install scikit-image pandas matplotlib tqdm  torchinfo tensorboard wandb scikit-learn

# Imports

In [1]:
import os
import torch
import psutil
from util import format
from torch.utils.data import DataLoader
from datasets import DeepLenseClassificationDataset
from torchinfo import summary
from util import run_experiment

# CONFIG

In [2]:
class CONFIG:
    BATCH_SIZE = 16
    
    # limit the data to prototype faster
    DATA_LIMIT = 100
    
    CLASSES = 3
    ROUND_NUMBER = 3
    DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    TASK_NAME = "DeepLense2024_task1"

In [3]:
memory_start = psutil.virtual_memory()
print(f"RAM used : {format(memory_start.used)}, RAM free : {format(memory_start.free)}")

RAM used : 2.9GB, RAM free : 2.94GB


# Loading the dataset

In [4]:
training_data = DeepLenseClassificationDataset(os.path.join(".", "Data", "dataset", "train"), data_limit=CONFIG.DATA_LIMIT, randomize_dataset=False,
                                               mean=0, std=1)
train_dataloader = DataLoader(training_data, batch_size=CONFIG.BATCH_SIZE, shuffle=True)


mean, std = training_data.mean, training_data.std

val_data = DeepLenseClassificationDataset(os.path.join(".", "Data", "dataset", "val"), data_limit=CONFIG.DATA_LIMIT, randomize_dataset=False,
                                               mean=0, std=1)
val_dataloader = DataLoader(val_data, batch_size=CONFIG.BATCH_SIZE, shuffle=True)

memory = psutil.virtual_memory()
print(f"RAM used : {format(memory.used)}, RAM free : {format(memory.free)}, dataset used : {format(memory_start.free - memory.free)}")

loading numpy:  22%|██▏       | 65/300 [00:00<00:00, 646.72it/s]

preprocessing: 100%|██████████| 300/300 [00:00<00:00, 14295.68it/s]

RAM used : 3.01GB, RAM free : 2.73GB, dataset used : 219.95MB


# Simple Model : Resnet18 based

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18

class CNNv1(nn.Module):
    def __init__(self):
        super(CNNv1, self).__init__()
        # the resnet model was trained on imagenet, it's not really applicable in this situation
        self.backend = resnet18(None)

        # make one channel 
        self.backend.conv1 = nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)

        # output 3 classes
        self.backend.fc = nn.Linear(512, CONFIG.CLASSES, bias=True)
        

    def forward(self, x):
        # Applying convolutional layers with ReLU and pooling
        x = self.backend(x)
        return x

# Create the CNN model instance
model = CNNv1().to(CONFIG.DEVICE)

/home/mlewand/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [6]:
# Get the sample batch to prototype with models
x, y = next(iter(train_dataloader))
id2name = [v.split("/")[-1] for v in training_data.class_folders] 

In [7]:
model(x.float().to(CONFIG.DEVICE)).shape

summary(model, input_shape=(CONFIG.BATCH_SIZE, 150, 150, 1))

Layer (type:depth-idx)                        Param #
CNNv1                                         --
├─ResNet: 1-1                                 --
│    └─Conv2d: 2-1                            3,136
│    └─BatchNorm2d: 2-2                       128
│    └─ReLU: 2-3                              --
│    └─MaxPool2d: 2-4                         --
│    └─Sequential: 2-5                        --
│    │    └─BasicBlock: 3-1                   73,984
│    │    └─BasicBlock: 3-2                   73,984
│    └─Sequential: 2-6                        --
│    │    └─BasicBlock: 3-3                   230,144
│    │    └─BasicBlock: 3-4                   295,424
│    └─Sequential: 2-7                        --
│    │    └─BasicBlock: 3-5                   919,040
│    │    └─BasicBlock: 3-6                   1,180,672
│    └─Sequential: 2-8                        --
│    │    └─BasicBlock: 3-7                   3,673,088
│    │    └─BasicBlock: 3-8                   4,720,640
│    └─AdaptiveA

In [15]:
import util
from importlib import reload

reload(util)

util.run_experiment(Model=CNNv1, 
               run_name="resnet18", 
               model_parameters={},
               epochs=30,
               learning_rate=1e-4,
               optimizer="Adam",
               train_dataloader=train_dataloader,
               val_dataloader=val_dataloader,
               savepath=os.path.join("models", "resnet18.pth"),
               base_lr=3e-4,
               max_lr=1e-3,
               scheduler_en=False,
               cfg=CONFIG,
               metric_keyword="accuracy",
               lr_steps=len(train_dataloader) // 2,
               WANDB_ON=False)

EPOCH : 0, average loss : 1.154781184698406: 100%|██████████| 19/19 [00:00<00:00, 35.19it/s] 
examples seen so far : 300, accuracy = 0.333, AUC ROC = 0.551: 100%|██████████| 19/19 [00:00<00:00, 79.13it/s]
examples seen so far : 300, accuracy = 0.333, AUC ROC = 0.49: 100%|██████████| 19/19 [00:00<00:00, 57.30it/s] 
EPOCH : 1, average loss : 0.7910586344568353: 100%|██████████| 19/19 [00:00<00:00, 48.89it/s]
examples seen so far : 300, accuracy = 0.423, AUC ROC = 0.649: 100%|██████████| 19/19 [00:00<00:00, 87.92it/s]
examples seen so far : 300, accuracy = 0.293, AUC ROC = 0.488: 100%|██████████| 19/19 [00:00<00:00, 108.44it/s]
EPOCH : 2, average loss : 0.38338028914050054: 100%|██████████| 19/19 [00:00<00:00, 52.95it/s]
examples seen so far : 300, accuracy = 0.65, AUC ROC = 0.872: 100%|██████████| 19/19 [00:00<00:00, 103.14it/s] 
examples seen so far : 300, accuracy = 0.31, AUC ROC = 0.489: 100%|██████████| 19/19 [00:00<00:00, 101.25it/s] 
EPOCH : 3, average loss : 0.0911007239238212: 10